# SynPlanner retrosynthetic planning

This tutorial presents a minimal example of retrosynthetic planning that can be done with SynPlanner. Here the Monte Carlo tree search is performed using a ranking policy network and the rollout function.

**Data used:**

* Reaction rules extracted from the curated USPTO database
* Building blocks from eMolecules and Sigma-Aldrich provided by the authors of the ASKCOS tool

Click the ▶ play button at the left of the **Installation** text below to install the application. The initial installation process takes about 5-7 minutes.

In [ ]:
#@title SynPlanner Installation
%%bash

git clone https://github.com/Laboratoire-de-Chemoinformatique/SynPlanner.git
cd SynPlanner/
pip install -q .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.7/547.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

fatal: destination path 'SynPlanner' already exists and is not an empty directory.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.
inflect 7.3.1 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.2.2 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 2.2.2 which is incompatible.


## Download input data

The input data will be downloaded from the [HuggingFace repository](https://huggingface.co/Laboratoire-De-Chemoinformatique/SynPlanner) to the current directory.

In [ ]:
from pathlib import Path
from synplan.utils.loading import download_all_data

# replace this path where you want to keep all your results
root_folder = Path(".").resolve()
root_folder.mkdir(exist_ok=True)

download_all_data(save_to=root_folder)

ranking_policy_weights = root_folder.joinpath("uspto/weights/ranking_policy_network.ckpt").resolve(strict=True)
reaction_rules_path = root_folder.joinpath("uspto/uspto_reaction_rules.pickle").resolve(strict=True)
building_blocks_path = root_folder.joinpath("building_blocks/building_blocks_em_sa_ln.smi").resolve(strict=True)

Fetching 25 files:   0%|          | 0/25 [00:00<?, ?it/s]

.gitignore:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

(…)sascore/targets_with_sascore_1.5_2.5.smi:   0%|          | 0.00/4.36k [00:00<?, ?B/s]

(…)sascore/targets_with_sascore_3.5_4.5.smi:   0%|          | 0.00/5.40k [00:00<?, ?B/s]

...

uspto_reaction_rules.pickle:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

filtering_policy_network.ckpt:   0%|          | 0.00/312M [00:00<?, ?B/s]

ranking_policy_network.ckpt:   0%|          | 0.00/164M [00:00<?, ?B/s]

value_network.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

Extracted molecules_for_filtering_policy_training.smi to /content/chembl
Extracted targets_for_value_network_training.smi to /content/chembl
Extracted building_blocks_em_sa_ln.sdf to /content/building_blocks
Extracted uspto_standardized.smi to /content/uspto
Extracted uspto_tutorial.smi to /content/tutorial
Extracted uspto_filtered.smi to /content/tutorial/data_curation
Extracted uspto_standardized.smi to /content/tutorial/data_curation
Extracted uspto_reaction_rules.pickle to /content/tutorial/rules_extraction


## Initializing the search tree

The search tree in SynPlanner is represented by `Tree` class, which requires:

* loaded building blocks provided as CGRtools `MolecularContainer` object
* loaded reaction rules provided as CGRtools `Reactor` objects
* loaded expansion function represented as trained policy network.
* tree configuration of SynPlanner `TreeConfig` object
* target molecule provided as CGRtools `MolecularContainer` object

### Loading building blocks

Building blocks can be loaded with the `load_building_blocks` function. They can be loaded from a `.pickle` file containing already standardized molecules as `MolecularContainer` or from a `.smi` file containing SMILES.

<div class="alert alert alert-warning">
**Warn**
    
The first loading of building blocks can be long, especially if they loaded from SMILES file.
</div>

In [ ]:
from synplan.utils.loading import load_building_blocks

building_blocks = load_building_blocks(building_blocks_path, standardize=False)

### Loading reaction rules

Reaction rules can be loaded with `load_reaction_rules` function that will automatically convert them to `Reactor` object of CGRtools. Currently, they are loaded from `.pickle` file, that was generated during reaction rule extraction. The loading from SMARTS would be added in the next versions.

For more information, please visit rules extraction tutorial.

In [ ]:
from synplan.utils.loading import load_reaction_rules

reaction_rules = load_reaction_rules(reaction_rules_path)

### Loading policy network

Policy function can be loaded directly from the weights file ends with `.ckpt`, as it includes all configuration for the policy network used for its training.

<div class="alert alert-info">
**Note**  
    
The `PolicyNetworkConfig` parameters are ignored when the `.ckpt` file is provided, as it already contains all parameter values.
</div>

For more info, please visit ranking policy training tutorial.

In [ ]:
from synplan.mcts.expansion import PolicyNetworkFunction
from synplan.utils.config import PolicyNetworkConfig

policy_config = PolicyNetworkConfig(weights_path=ranking_policy_weights)
policy_network = PolicyNetworkFunction(policy_config=policy_config)

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.9.5 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint uspto/weights/ranking_policy_network.ckpt`


### Configuring search tree

The tree search is configured using the `TreeConfig` class. Key parameters include:

| Parameter            | Description                                                                                                      |
|----------------------|------------------------------------------------------------------------------------------------------------------|
| **max_iterations**   | Defines the total number of iterations the algorithm will perform, essentially setting a limit on how many times the tree search loop can run. Default is `100` |
| **max_tree_size**    | Sets the upper limit on the total number of nodes that can exist in the search tree, controlling the size and complexity of the tree. |
| **max_time**         | Specifies a time limit for the algorithm's execution, measured in seconds. This prevents the search from running indefinitely. |
| **max_depth**        | Determines the maximum depth of the tree, effectively controlling how far the search can go from the root node. The default maximum depth is 6 levels. |
| **ucb_type**         | Chooses the type of Upper Confidence Bound (UCB) algorithm used in the search. Options include `puct` (predictive UCB), `uct` (standard UCB), and "value". |
| **c_ucb**            | This is the exploration-exploitation balance coefficient in UCB, which influences how much the algorithm favors exploration of new paths versus exploitation of known paths. |
| **backprop_type**    | Selects the backpropagation method used during the search. Options are `muzero` (model-based approach) and `cumulative` (cumulative reward approach). |
| **search_strategy**  | Determines the strategy for navigating the tree. Options are "expansion_first" (prioritizing the expansion of new nodes) and `evaluation_first` (prioritizing the evaluation of existing nodes). |
| **evaluation_agg**   | This setting determines how the evaluation scores are aggregated. Options are `max` (using the maximum score) and `average` (using the average score). |
| **evaluation_type**  | Defines the method used for node evaluation. Options include `random` (random evaluations), `rollout` (using rollout simulations), and `gcn` (graph convolutional networks). |
| **init_node_value**  | Sets the initial value for newly created nodes in the tree. This can impact how nodes are prioritized during the search. |
| **epsilon**          | This parameter is used in the epsilon-greedy strategy during node selection, representing the probability of choosing a random action for exploration. A higher value leads to more exploration. |
| **exclude_small**    | A boolean setting that, when `True`, excludes small molecules from the search, typically to focus on more complex molecules. |
| **min_mol_size**     | Defines the minimum size of a molecule (in terms of the number of heavy atoms count) to be considered in the search. Molecules smaller than this threshold are typically considered as readily available building blocks. |
| **silent**           | When set to `True`, this option suppresses the progress output of the tree search, keeping the output clean and focused. |


`TreeConfig` can be specified manually or loaded from `.yaml` config file:

In [ ]:
from synplan.utils.config import TreeConfig

tree_config = TreeConfig(
    search_strategy="expansion_first",
    evaluation_type="rollout",
    max_iterations=50,
    max_time=120,
    max_depth=9,
    min_mol_size=1,
    init_node_value=0.5,
    ucb_type="uct",
    c_ucb=0.1,
)

### Loading target molecule

The target molecule need to be loaded as `MolecularContainer` and **must be standardized**. The easiest way is to use `smiles` function availible in CGRtools.

For the search with multiple molecules, please visit advanced retrosynthetic planning tutorial.

In [ ]:
from synplan.chem.utils import mol_from_smiles

# let's take capivasertib used as anti-cancer medication for the treatment
# of breast cancer and approved by FDA in 2023
example_smiles = "NC1(C(=O)N[C@@H](CCO)c2ccc(Cl)cc2)CCN(c2nc[nH]c3nccc2-3)CC1"

target_molecule = mol_from_smiles(example_smiles, clean2d=True, standardize=True, clean_stereo=True)

### Initialising tree

Next, we initialise the `Tree` object, providing target molecule and loaded reaction rules, building blocks and expansion/evaluation functions.

The `evaluation_function` parameter expects `ValueNetwork` object. If `tree_config.evaluation_type` parameter will be `random` or `rollout`, the `evaluation_function` will be ignored.

In [ ]:
from synplan.mcts.tree import Tree

tree = Tree(
    target=target_molecule,
    config=tree_config,
    reaction_rules=reaction_rules,
    building_blocks=building_blocks,
    expansion_function=policy_network,
    # you can also specify evaluation_function=ValueNetwork(...), by default it is None
)

## Running the search

The `Tree` object is iterable, like `List` in Python. Each iteration represents one MCTS iteration, after which 2 values are returned:

* `solved` is boolean, if `True` then a successful route was found during the current iteration.
* `node_id` is a list of indices of leaf nodes generated during the current iteration.

Once search is finished, the tree will print some stats:

In [ ]:
tree_solved = False
for solved, node_id in tree:
    if solved:
        tree_solved = True
tree

  0%|          | 0/50 [00:00<?, ?it/s]

Tree for: c1cc(ccc1Cl)C(CCO)NC(C2(CCN(CC2)c3c4cc[nH]c4ncn3)N)=O
Time: 31.5 seconds
Number of nodes: 1000
Number of iterations: 50
Number of visited nodes: 50
Number of found routes: 12

## Visualizing retrosynthesis routes

After the tree search is complete, we can visualize the found retrosynthesis paths. The visualization uses the `get_route_svg` function from SynPlanner visualization interface.

In [ ]:
from IPython.display import SVG, display
from synplan.utils.visualisation import get_route_svg

for n, node_id in enumerate(tree.winning_nodes):
    print(
        f"-------- Path starts from node #{node_id} with total route score {tree.route_score(node_id)} --------"
    )
    display(SVG(get_route_svg(tree, node_id)))
    if n == 3:
        break

-------- Path starts from node #85 with total route score 0.11346666666666666 --------


-------- Path starts from node #86 with total route score 0.11346666666666666 --------


-------- Path starts from node #88 with total route score 0.11346666666666666 --------


-------- Path starts from node #85 with total route score 0.11346666666666666 --------
